# Import Libraries

In [ ]:
from openpyxl import load_workbook, Workbook
import re
import os
import pandas as pd

# Prepare CSV

## Sort Chunk 5 by ID

In [1]:
from openpyxl import load_workbook, Workbook
import re

# File names
input_file = "Chunk 5.xlsx"
output_file = "Chunk 5 Fully Sorted.xlsx"

# Custom sorting function for IDs
def custom_sort_key(id_str):
    # Regex to split numeric and alphabetical parts
    match = re.match(r"(.+?)_(\d+)([a-z]*)$", id_str)
    if match:
        prefix, number, suffix = match.groups()
        return (prefix, int(number), suffix)
    return (id_str, 0, "")

try:
    # Load the workbook and get the active worksheet
    wb = load_workbook(input_file)
    ws = wb.active

    # Read data from the worksheet into a list of dictionaries
    data = []
    header = [cell.value for cell in ws[1]]  # First row is the header
    for row in ws.iter_rows(min_row=2, values_only=True):  # Iterate over rows (skip header)
        data.append(dict(zip(header, row)))

    # Sort the data based on the ID column using the custom sort key
    sorted_data = sorted(data, key=lambda x: custom_sort_key(str(x['ID'])))

    # Create a new workbook and worksheet
    new_wb = Workbook()
    new_ws = new_wb.active

    # Write the header
    new_ws.append(header)

    # Write the sorted data
    for row in sorted_data:
        new_ws.append([row[col] for col in header])

    # Save the new workbook
    new_wb.save(output_file)
    print(f"Rows sorted and saved as '{output_file}'.")
except Exception as e:
    print(f"Error processing file: {e}")

Rows sorted and saved as 'Chunk 5 Fully Sorted.xlsx'.


## Update Chunk 5 IDs 
Convert all lettered IDs to proper number-only IDs

In [ ]:
# File names
input_file = "Chunk 5 Fully Sorted.xlsx"
output_file = "Chunk 5 IDs Updated.xlsx"

# Custom sorting function for IDs
def custom_sort_key(id_str):
    """
    Extract prefix, numeric part, and suffix to sort numerically and alphabetically.
    """
    match = re.match(r"(.+?)_(\d+)([a-z]*)$", id_str)
    if match:
        prefix, number, suffix = match.groups()
        return (prefix, int(number), suffix or "")  # Treat numeric part as integer
    return (id_str, 0, "")  # Default for unmatched IDs

# Function to process and update IDs
def process_ids(data):
    updated_data = []
    existing_ids = set()  # Track IDs that have been processed

    for i, row in enumerate(data):
        current_id = row['ID']
        match = re.match(r"(.+?)_(\d+)([a-z]*)$", current_id)
        if match:
            base_id, number, suffix = match.groups()
            number = int(number)  # Convert numeric part to integer

            # If there's a suffix, adjust the ID and shift subsequent numbers
            if suffix:
                new_id = f"{base_id}_{number + 1:03}"
                while new_id in existing_ids:  # Ensure uniqueness locally
                    number += 1
                    new_id = f"{base_id}_{number + 1:03}"
                row['ID'] = new_id
                existing_ids.add(new_id)

                # Shift subsequent IDs if necessary
                for j in range(i + 1, len(data)):
                    next_row_id = data[j]['ID']
                    next_match = re.match(r"(.+?)_(\d+)([a-z]*)$", next_row_id)
                    if next_match:
                        next_base_id, next_number, next_suffix = next_match.groups()
                        if next_base_id == base_id and int(next_number) <= number:
                            number += 1
                            updated_id = f"{base_id}_{number + 1:03}"
                            data[j]['ID'] = updated_id
                            existing_ids.add(updated_id)
            else:
                existing_ids.add(current_id)

        updated_data.append(row)

    return updated_data

try:
    # Load the workbook and worksheet
    wb = load_workbook(input_file)
    ws = wb.active

    # Read data from the worksheet into a list of dictionaries
    data = []
    header = [cell.value for cell in ws[1]]  # First row is the header
    for row in ws.iter_rows(min_row=2, values_only=True):  # Iterate over rows (skip header)
        data.append(dict(zip(header, row)))

    # Sort the data based on the ID column using the custom sort key
    sorted_data = sorted(data, key=lambda x: custom_sort_key(str(x['ID'])))

    # Process and update IDs
    updated_data = process_ids(sorted_data)

    # Create a new workbook and worksheet
    new_wb = Workbook()
    new_ws = new_wb.active

    # Write the header
    new_ws.append(header)

    # Write the updated data
    for row in updated_data:
        new_ws.append([row[col] for col in header])

    # Save the new workbook
    new_wb.save(output_file)
    print(f"Updated IDs saved as '{output_file}'.")
except Exception as e:
    print(f"Error processing file: {e}")

Updated IDs saved as 'Chunk 5 IDs Updated.xlsx'.


## Remove unnecessary "gaps" in IDs. 
For eg if there's a 51 and then directly a 53, the latter must be changed to 52

In [3]:
# File names
input_file = "Chunk 5 Fully Sorted.xlsx"
output_file = "Chunk 5 IDs Unique.xlsx"

# Custom sorting function for IDs
def custom_sort_key(id_str):
    """
    Extract prefix, numeric part, and suffix to sort numerically and alphabetically.
    """
    match = re.match(r"(.+?)_(\d+)([a-z]*)$", id_str)
    if match:
        prefix, number, suffix = match.groups()
        return (prefix, int(number), suffix or "")  # Treat numeric part as integer
    return (id_str, 0, "")  # Default for unmatched IDs

# Function to process and update IDs to ensure uniqueness
def process_ids(data):
    existing_ids = set()  # Track already-used IDs
    for i, row in enumerate(data):
        current_id = row['ID']
        match = re.match(r"(.+?)_(\d+)([a-z]*)$", current_id)
        if match:
            base_id, number, suffix = match.groups()
            number = int(number)  # Convert numeric part to integer

            # Generate a new ID if there is a suffix or conflict
            if suffix or current_id in existing_ids:
                new_id = f"{base_id}_{number:03}"
                while new_id in existing_ids:  # Ensure uniqueness
                    number += 1
                    new_id = f"{base_id}_{number:03}"
                row['ID'] = new_id  # Update the row's ID
                existing_ids.add(new_id)  # Add to the set of used IDs
            else:
                existing_ids.add(current_id)  # Add original ID if no conflict

    return data

try:
    # Load the workbook and worksheet
    wb = load_workbook(input_file)
    ws = wb.active

    # Read data from the worksheet into a list of dictionaries
    data = []
    header = [cell.value for cell in ws[1]]  # First row is the header
    for row in ws.iter_rows(min_row=2, values_only=True):  # Iterate over rows (skip header)
        data.append(dict(zip(header, row)))

    # Sort the data based on the ID column using the custom sort key
    sorted_data = sorted(data, key=lambda x: custom_sort_key(str(x['ID'])))

    # Process and update IDs to ensure uniqueness
    updated_data = process_ids(sorted_data)

    # Create a new workbook and worksheet
    new_wb = Workbook()
    new_ws = new_wb.active

    # Write the header
    new_ws.append(header)

    # Write the updated data
    for row in updated_data:
        new_ws.append([row[col] for col in header])

    # Save the new workbook
    new_wb.save(output_file)
    print(f"Updated IDs saved as '{output_file}'.")
except Exception as e:
    print(f"Error processing file: {e}")

Updated IDs saved as 'Chunk 5 IDs Unique.xlsx'.


# Extracting Reviews

Extract reviews using consecutive metadata rows as anchors

In [24]:
import os
import pandas as pd
import re

# Define paths
chunk_file = "Chunk 5 IDs Unique.xlsx"           # Excel file with metadata
text_files_folder = "Files"           # Input folder containing text files
output_folder = "Reviews"             # Output folder for extracted reviews

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the Excel file
df = pd.read_excel(chunk_file, dtype=str)

# Helper function to clean and normalize text
def clean_text(text):
    return re.sub(r'\s+', ' ', str(text).strip())

# Helper function to check for metadata match
def is_match(line, value):
    return value and clean_text(value).lower() in clean_text(line).lower()

# Function to validate Price, Publisher, and Year in a block of lines
def validate_metadata(lines, start_idx, price_list, publisher, year):
    for offset in range(4):  # Check current line + next 3 lines
        if start_idx + offset < len(lines):
            nearby_line = clean_text(lines[start_idx + offset]).lower()

            # Check for any price match
            if any(price.strip() in nearby_line for price in price_list):
                return True

            # Check for Publisher-Year combo
            if publisher and year and f"{publisher.lower()}, {year}" in nearby_line:
                return True

            # Check year alone
            if year and year in nearby_line:
                return True
    return False

# Function to find metadata position
def find_metadata(lines, row, start_idx):
    title = clean_text(row['Title']).lower()
    author = clean_text(row['Author']).lower()
    price_list = clean_text(row['Price']).split(',') if pd.notna(row['Price']) else []  # Handle NaN safely
    publisher = clean_text(row['Publisher']) if pd.notna(row['Publisher']) else ""
    year = clean_text(row['Year']) if pd.notna(row['Year']) else ""

    # Split author into components for partial matching
    author_parts = [part.strip() for part in author.split(',')] if author else []

    idx = start_idx
    while idx < len(lines):
        line = clean_text(lines[idx]).lower()
        next_line = clean_text(lines[idx + 1]).lower() if idx + 1 < len(lines) else ""

        # Check for Title match
        title_match = title in line or title in next_line

        # Check for Author match (partial matching with any component)
        author_match = any(part in line or part in next_line for part in author_parts)

        # Proceed if Title or Author matches and validate with metadata
        if title_match or author_match:
            # Ensure both Title and Author are present within the same or next line
            if title_match and author_match:
                if validate_metadata(lines, idx, price_list, publisher, year):
                    return idx
        idx += 1

    return -1

# Track the total number of "Review could not be extracted" cases
empty_review_count = 0

# Process each row
for i, row in df.iterrows():
    book_id = row['ID']
    title = clean_text(row['Title'])
    author = clean_text(row['Author']) if pd.notna(row['Author']) else ""
    price = clean_text(row['Price']) if pd.notna(row['Price']) else ""
    pages = clean_text(row['Pages']) if pd.notna(row['Pages']) else ""
    grade_years = clean_text(str(row['Grade'])) if pd.notna(row['Grade']) else ""
    publisher = clean_text(row['Publisher']) if pd.notna(row['Publisher']) else ""
    year = clean_text(row['Year']) if pd.notna(row['Year']) else ""

    # Extract the filename from the ID
    volume_file = "_".join(book_id.split("_")[:3]) + ".txt"
    volume_file_path = os.path.join(text_files_folder, volume_file)

    if not os.path.exists(volume_file_path):
        print(f"Text file not found for ID {book_id}. Skipping...")
        continue

    # Read the text file
    with open(volume_file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Find the start index for current metadata
    start_index = find_metadata(lines, row, 0)
    if start_index == -1:
        print(f"Start not found for ID {book_id}. Saving placeholder.")
        empty_review_count += 1  # Increment counter if no valid review content
        with open(os.path.join(output_folder, f"{book_id}.txt"), 'w', encoding='utf-8') as file:
            file.write("Review could not be extracted.")
        continue

    # Find the end index based on next row
    end_index = len(lines)
    if i + 1 < len(df):
        next_row = df.iloc[i + 1]
        next_title, next_author = map(
            lambda x: clean_text(x) if pd.notna(x) else "", 
            [next_row['Title'], next_row['Author']]
        )
        for idx in range(start_index + 2, len(lines)):
            line = lines[idx]
            # Check if the next metadata (title or author) is found
            if is_match(line, next_title) or is_match(line, next_author):
                end_index = idx  # Stop at the start of the next metadata line
                break

    # Extract review text
    review_lines = lines[start_index + 1:end_index]

    # Remove lines containing specific metadata (only once per category)
    cleaned_lines = []
    removed_price = removed_pages = removed_grade = removed_publisher = False

    for line in review_lines:
        line_str = clean_text(str(line))  # Ensure all lines are strings
        if not removed_price and is_match(line_str, price):
            removed_price = True
            continue
        if not removed_pages and is_match(line_str, pages):
            removed_pages = True
            continue
        if not removed_grade and is_match(line_str, grade_years):
            removed_grade = True
            continue
        if not removed_publisher and is_match(line_str, publisher):
            removed_publisher = True
            continue
        cleaned_lines.append(line)

    # Combine cleaned lines
    review_text = "".join(cleaned_lines).strip()
    if not review_text:
        empty_review_count += 1  # Increment counter if no valid review content
        review_text = "Review could not be extracted."
        print(f"Start not found for ID {book_id}. Saving placeholder.")

    # Save the review text
    review_file_path = os.path.join(output_folder, f"{book_id}.txt")
    with open(review_file_path, 'w', encoding='utf-8') as file:
        file.write(review_text)

    print(f"Processed review for ID {book_id}")

# Print summary of empty reviews
print(f"\nTotal 'Review could not be extracted' files: {empty_review_count}")

# Save updated DataFrame
df.to_excel("Chunk 5 parsed.xlsx", index=False)


Processed review for ID 19_(01)_1965_001
Processed review for ID 19_(01)_1965_002
Processed review for ID 19_(01)_1965_003
Processed review for ID 19_(01)_1965_004
Processed review for ID 19_(01)_1965_005
Processed review for ID 19_(01)_1965_006
Processed review for ID 19_(01)_1965_007
Processed review for ID 19_(01)_1965_008
Processed review for ID 19_(01)_1965_009
Processed review for ID 19_(01)_1965_010
Processed review for ID 19_(01)_1965_012
Processed review for ID 19_(01)_1965_013
Processed review for ID 19_(01)_1965_014
Processed review for ID 19_(01)_1965_015
Processed review for ID 19_(01)_1965_016
Processed review for ID 19_(01)_1965_017
Processed review for ID 19_(01)_1965_018
Processed review for ID 19_(01)_1965_019
Processed review for ID 19_(01)_1965_020
Processed review for ID 19_(01)_1965_021
Processed review for ID 19_(01)_1965_022
Processed review for ID 19_(01)_1965_023
Processed review for ID 19_(01)_1965_024
Processed review for ID 19_(01)_1965_025
Processed review

# Processing Reviews

## Remove lines which only contain numbers or any mention of "ISBN"

In [25]:
# Define input and output folders
input_folder = "Reviews"  # Input folder containing original review text files
output_folder = "Reviews Processed"  # Output folder for processed review text files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Regex patterns
numeric_only_regex = re.compile(r"^\s*[0-9\-\.\,]+\s*$")  # Lines with only numbers, -, ., or ,
isbn_regex = re.compile(r"\bISBN\b", re.IGNORECASE)  # Lines containing 'ISBN' (case-insensitive)

# Function to process a single file
def process_file(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    cleaned_lines = []
    modified = False

    for line in lines:
        # Check if the line matches either numeric-only or contains 'ISBN'
        if numeric_only_regex.match(line) or isbn_regex.search(line):
            modified = True  # Mark the file as modified
            continue  # Skip this line
        cleaned_lines.append(line)

    # Save the cleaned file
    with open(output_path, 'w', encoding='utf-8') as file:
        file.writelines(cleaned_lines)

    return modified

# Track modified files
modified_files = []

# Process each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename)

        # Process the file and check if it was modified
        if process_file(input_file_path, output_file_path):
            modified_files.append(filename)

# Print the summary
print("Processing complete. Processed files saved in 'Reviews Processed1'.")
print(f"Total files with deletions: {len(modified_files)}")
if modified_files:
    print("Files with lines removed:")
    for file in modified_files:
        print(f" - {file}")
else:
    print("No files had lines removed.")

Processing complete. Processed files saved in 'Reviews Processed1'.
Total files with deletions: 2471
Files with lines removed:
 - 19_(01)_1965_092.txt
 - 19_(02)_1965_021.txt
 - 19_(02)_1965_036.txt
 - 19_(03)_1965_020.txt
 - 19_(04)_1965_017.txt
 - 19_(04)_1965_020.txt
 - 19_(04)_1965_034.txt
 - 19_(04)_1965_059.txt
 - 19_(04)_1965_064.txt
 - 19_(05)_1966_019.txt
 - 19_(05)_1966_032.txt
 - 19_(06)_1966_034.txt
 - 19_(06)_1966_035.txt
 - 19_(07)_1966_031.txt
 - 19_(07)_1966_033.txt
 - 19_(07)_1966_057.txt
 - 19_(08)_1966_002.txt
 - 19_(08)_1966_028.txt
 - 19_(08)_1966_035.txt
 - 19_(09)_1966_006.txt
 - 19_(09)_1966_017.txt
 - 19_(10)_1966_026.txt
 - 19_(10)_1966_032.txt
 - 19_(10)_1966_040.txt
 - 19_(10)_1966_047.txt
 - 19_(10)_1966_053.txt
 - 19_(11)_1966_016.txt
 - 19_(11)_1966_023.txt
 - 19_(11)_1966_033.txt
 - 19_(11)_1966_066.txt
 - 19_(11)_1966_067.txt
 - 19_(11)_1966_069.txt
 - 20_(01)_1966_001.txt
 - 20_(01)_1966_002.txt
 - 20_(01)_1966_006.txt
 - 20_(01)_1966_010.txt
 - 20_(01

## Identify presence of extraneous books and remove them from each review file

#### First pass with Author + Coded Symbol + "illus. by"

In [26]:
# Define input and output folders
input_folder = "Reviews Processed"  # Folder containing original review text files
output_folder = "Reviews Processed1"  # Folder to save processed reviews

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Regex patterns
author_regex = re.compile(
    r"^\s*([A-Z][A-Za-z'-]+(?:\s[A-Za-z'-]+)*,)"  # LastName or multi-word LastName (e.g., "De Leeuw,")
    r"\s([A-Z][A-Za-z'-]*"                       # FirstName (e.g., Adele)
    r"(?:\s[A-Z]\.)*"                            # Optional middle initials (e.g., "A.")
    r"(?:\s[A-Za-z'-]+)?)"                       # Optional middle/compound name (e.g., Adele Louise)
)
price_regex = re.compile(r"(\$\d+\.?\d*)")
pages_regex = re.compile(r"\b(\d{1,3}p)\b")
publisher_year_regex = re.compile(r"\.\s+([^.,]+),\s(\d{4})")  # Matches: Publisher, 1958
illus_by_regex = re.compile(r"illus\. by", re.IGNORECASE)  # Match 'illus. by' case-insensitively

# Coded symbols to match (case insensitive, whole line)
coded_symbols = ["R*", "R", "Ad", "M", "NR", "SpC", "SpR"]
coded_symbol_regex = re.compile(r"^\s*(" + "|".join(coded_symbols) + r")\s*$", re.IGNORECASE)

# Function to process a single file
def process_file(input_path, output_path, filename):
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    output_lines = []
    skip_flag = False  # Flag to stop processing after identifying conditions
    identified_lines = []  # Track identified lines for reporting

    for i, line in enumerate(lines):
        if i < 4:  # Skip the first 4 lines
            output_lines.append(line)
            continue
        
        if not skip_flag:
            # # Step 1: Check for coded symbols (as the ONLY content on a line)
            # if coded_symbol_regex.match(line):
            #     skip_flag = True  # Start skipping everything after this line
            #     identified_lines.append(line.strip())
            #     break  # Stop further checks on this line

            # Step 2: Check for 'illus. by' and validate with price/pages/publisher
            if illus_by_regex.search(line):
                valid_match = False
                for offset in range(0, 3):  # Current line + next 2 lines
                    next_idx = i + offset
                    if next_idx < len(lines):
                        combined_line = lines[next_idx]
                        if (price_regex.search(combined_line) or
                            pages_regex.search(combined_line) or
                            publisher_year_regex.search(combined_line)):
                            valid_match = True
                            break
                
                if valid_match:  # Trigger skip if conditions are met
                    skip_flag = True
                    identified_lines.append(line.strip())
                    break

            # Step 3: Check for Author line and validate with price/pages/publisher
            if author_regex.match(line):
                valid_match = False
                for offset in range(0, 2):  # Current line + next line
                    next_idx = i + offset
                    if next_idx < len(lines):
                        combined_line = lines[next_idx]
                        if (price_regex.search(combined_line) or
                            pages_regex.search(combined_line) or
                            publisher_year_regex.search(combined_line)):
                            valid_match = True
                            break

                if valid_match:
                    skip_flag = True
                    identified_lines.append(line.strip())
                    break

        # Append lines if no skip condition is triggered
        if not skip_flag:
            output_lines.append(line)

    # Write cleaned content to the output file
    with open(output_path, 'w', encoding='utf-8') as file:
        file.writelines(output_lines)

    return identified_lines  # Return all identified lines for reporting

# Track identified lines and files with removals
files_with_removals = 0
all_removed_lines = {}

# Process all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename)

        removed_lines = process_file(input_file_path, output_file_path, filename)
        if removed_lines:
            files_with_removals += 1
            all_removed_lines[filename] = removed_lines

# Final summary
print(f"Processing complete. Cleaned files are saved in '{output_folder}'.")
print(f"Number of files where lines were identified and removed: {files_with_removals}\n")

# Print all identified lines
print("Identified lines removed from each file:")
for file, lines in all_removed_lines.items():
    print(f"\nFile: {file}")
    for line in lines:
        print(f" - {line}")

Processing complete. Cleaned files are saved in 'Reviews Processed1'.
Number of files where lines were identified and removed: 630

Identified lines removed from each file:

File: 19_(03)_1965_020.txt
 - L'Engle, Madeline. Camilla. T. Y. Crowell, 1965. 282p. $4.50.

File: 19_(04)_1965_017.txt
 - Ferra-Mikura, Vera. Twelve People Are Not a Dozen; tr. by Catherine Hutter;

File: 19_(04)_1965_034.txt
 - Manning-Sanders, Ruth. A Book of Dwarfs; drawings by Robin Jacques. Dutton,

File: 19_(07)_1966_013.txt
 - De Regniers, Beatrice Schenk. How Joe the Bear and Sam the Mouse Got Together;

File: 19_(07)_1966_033.txt
 - McGovern, Ann. Runaway Slave; The Story of Harriet Tubman; pictures by R. M.

File: 19_(08)_1966_028.txt
 - La Fontaine, Jean de. The Rich Man and the Shoe-Maker; illus. by Brian Wild-

File: 19_(09)_1966_017.txt
 - De Leeuw, Adele Louise. Miss Fix-It. Macmillan, 1966. 229p. Trade ed. $3.50;

File: 19_(10)_1966_032.txt
 - McNair, Kate. A Sense of Magic. Chilton, 1965. 218p. $3

#### Second pass with publisher/price/pages

In [27]:
# Define input and output folders
input_folder = "Reviews Processed1"  # Input folder from first pass
output_folder = "Reviews Processed2"  # Output folder for second pass

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Regex patterns for price, pages, and publisher
price_regex = re.compile(r"(\$\d+\.?\d*)")
pages_regex = re.compile(r"\b(\d{1,3}p)\b")
publisher_year_regex = re.compile(r"\.\s+([^.,]+),\s(\d{4})")

# Function to check for valid combos
def has_valid_combo(lines, start_idx):
    conditions = {"price": False, "pages": False, "publisher": False}

    for offset in range(0, 3):  # Check current line + next 2 lines
        if start_idx + offset < len(lines):
            combined_line = lines[start_idx + offset]
            if price_regex.search(combined_line):
                conditions["price"] = True
            if pages_regex.search(combined_line):
                conditions["pages"] = True
            if publisher_year_regex.search(combined_line):
                conditions["publisher"] = True
        if sum(conditions.values()) >= 2:
            return True
    return False

# Function to process files
def process_file(input_path, output_path, filename):
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    output_lines = []
    skip_flag = False
    identified_lines = []

    for i, line in enumerate(lines):
        if i < 6:  # Keep the first 4 lines as they are
            output_lines.append(line)
            continue

        if not skip_flag:
            if (price_regex.search(line) or pages_regex.search(line) or publisher_year_regex.search(line)):
                if has_valid_combo(lines, i):
                    skip_flag = True
                    identified_lines.append(line.strip())
                    break

        if not skip_flag:
            output_lines.append(line)

    with open(output_path, 'w', encoding='utf-8') as file:
        file.writelines(output_lines)

    return identified_lines

# Second Pass Execution
files_with_removals = 0
all_removed_lines = {}

for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename)

        removed_lines = process_file(input_file_path, output_file_path, filename)
        if removed_lines:
            files_with_removals += 1
            all_removed_lines[filename] = removed_lines

print("Second Pass Complete. Files saved to 'Reviews Processed2'.")
print(f"Files cleaned in Second Pass: {files_with_removals}\n")

# Print identified lines removed for each file
print("Identified lines removed from each file:")
for file, lines in all_removed_lines.items():
    print(f"\nFile: {file}")
    for line in lines:
        print(f" - {line}")

Second Pass Complete. Files saved to 'Reviews Processed1'.
Files cleaned in Second Pass: 150

Identified lines removed from each file:

File: 19_(02)_1965_036.txt
 - 1964. 160p. illus. $2.95.

File: 19_(05)_1966_032.txt
 - Life Magazine. Early Man; by F. Clark Howell and the editors of Life. Time, 1965.

File: 19_(05)_1966_035.txt
 - Mariana. The Journey of Bangwell Putt. Lothrop, 1965. 35p. illus. $3.50.

File: 19_(07)_1966_031.txt
 - itors of Life. Time, 1964. 200p. illus. $3.95.

File: 19_(08)_1966_035.txt
 - Rhymes. Watts, 1964. 80p. illus. $4.95.

File: 19_(09)_1966_017.txt
 - 69p. $3.50.

File: 19_(10)_1966_032.txt
 - Lightner,A. M. The Galactic Troubadours. Norton, 1965. 237p. Trade ed. $3.75;

File: 20_(03)_1966_001.txt
 - guerite Northrup. Metropolitan Museum of Art, 1966. 32p. $3.75.

File: 20_(04)_1966_013.txt
 - 210p. $3.95.

File: 20_(04)_1966_024.txt
 - Tischler. Atheneum, 1965.

File: 20_(07)_1967_035.txt
 - World. Simon and Schuster, 1966. 214p. illus. $4.95.

File: 20_

### Remove lines with $ symbol, specifically those containing "Library" or "Trade" or "Paperback" or "Hardbound"

In [28]:
# Define input and output folders
input_folder = "Reviews Processed2"  # Folder containing processed review files
output_folder = "Reviews Processed3"  # Folder to save further processed reviews

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Patterns
keywords = ["library", "trade", "paperback", "hardbound"]
price_pattern = re.compile(r"\$\d+\.?\d*", re.IGNORECASE)  # Match any $ value like $12.15
ed_price_pattern = re.compile(r"\bed\.,?\s*\$\d+\.?\d*", re.IGNORECASE)  # Match "ed., $2.50"
edition_price_pattern = re.compile(r"\bedition,?\s*\$\d+\.?\d*", re.IGNORECASE)  # Match "edition, $2.50"
net_price_pattern = re.compile(r"\$\d+\.?\d*\s*net\.?", re.IGNORECASE)  # Match "$2.63 net."
grade_years_regex = re.compile(r"^(\d{1,2}-\d{1,2}|[A-Z]-\d{1,2}|.*(?:Gr\.|yrs\.).*)$")  # Match grade years

# Function to check for keyword and price combination
def contains_keyword_and_price(line):
    lower_line = line.lower()  # Case-insensitive keyword search
    if any(keyword in lower_line for keyword in keywords) and price_pattern.search(line):
        return True
    return False

# Function to check for "ed., $<amount>" pattern
def contains_ed_price(line):
    return bool(ed_price_pattern.search(line))

# Function to check for "edition, $<amount>" pattern
def contains_edition_price(line):
    return bool(edition_price_pattern.search(line))

# Function to check for "$<amount> net" pattern
def contains_net_price(line):
    return bool(net_price_pattern.search(line))

# Function to check if a line ONLY contains a number or price
def contains_only_number_or_price(line):
    stripped_line = line.strip()
    return bool(re.fullmatch(r"\d+|\$\d+\.?\d*", stripped_line))

# Function to check if a line ONLY contains grade years
def contains_only_grade_years(line):
    return bool(grade_years_regex.fullmatch(line.strip()))

# Function to process a single file
def process_file(input_path, output_path, filename):
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    cleaned_lines = []
    flagged_cases = []  # To record lines removed for debugging

    for idx, line in enumerate(lines):
        # Check conditions to remove lines
        if (contains_keyword_and_price(line) or contains_ed_price(line) or
            contains_edition_price(line) or contains_net_price(line) or
            contains_only_number_or_price(line) or contains_only_grade_years(line)):
            
            # Flag the line for debugging if not the first line
            if idx > 0:
                flagged_cases.append((filename, idx + 1, line.strip()))
            continue  # Skip this line (remove it)

        # Keep the line if no condition is met
        cleaned_lines.append(line)

    # Write cleaned content to the output file (remove blank lines)
    with open(output_path, 'w', encoding='utf-8') as file:
        file.writelines([line for line in cleaned_lines if line.strip()])

    # Print flagged cases
    for case in flagged_cases:
        print(f"File: {case[0]} | Line {case[1]}: {case[2]}")

# Process all files in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename)

        process_file(input_file_path, output_file_path, filename)

print("Processing complete. Cleaned files are saved in 'Reviews Processed3'.")

File: 19_(01)_1965_015.txt | Line 2: $3.50; Library ed. $3.27 net.
File: 19_(01)_1965_029.txt | Line 2: Library ed. $2.19 net.
File: 19_(02)_1965_002.txt | Line 2: tage Junior Library Series) Trade ed. $3.95; Library ed. $3.79 net.
File: 19_(02)_1965_003.txt | Line 2: illus. (American Heritage Junior Library Series) Trade ed. $3.95; Li-
File: 19_(02)_1965_003.txt | Line 3: brary ed. $3.79 net.
File: 19_(02)_1965_014.txt | Line 3: yrs.
File: 19_(03)_1965_015.txt | Line 2: yrs.
File: 19_(03)_1965_019.txt | Line 3: yrs.
File: 19_(03)_1965_032.txt | Line 3: yrs.
File: 19_(03)_1965_036.txt | Line 2: yrs.
File: 19_(04)_1965_014.txt | Line 3: yrs.
File: 19_(04)_1965_025.txt | Line 2: Trade ed. $1; Library ed.
File: 19_(04)_1965_025.txt | Line 3: $2.19 net.
File: 19_(04)_1965_025.txt | Line 5: yrs.
File: 19_(04)_1965_029.txt | Line 2: Trade ed. $3.25; Library ed.
File: 19_(04)_1965_029.txt | Line 3: $2.93 net.
File: 19_(04)_1965_060.txt | Line 3: yrs.
File: 19_(04)_1965_063.txt | Line 3: yrs.


### Remove lines containing only review codes or price

coded_symbols = ["R*", "R", "Ad", "M", "NR", "SpC", "SpR"]

In [29]:
# Define the input and output folders
input_folder = "Reviews Processed3"  # Input folder
output_folder = "Reviews Processed4"  # Output folder for cleaned files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List of coded symbols to identify and remove
coded_symbols = ["R*", "R", "Ad", "M", "NR", "SpC", "SpR"]

# Updated regex pattern to match lines that contain ONLY a price with optional punctuation
price_regex = re.compile(r"^\s*\$\d+(\.\d{2})?[\s\.\!]*$")  # Matches "$3.25.", "$10", etc.

# Track updated files and unchanged files
updated_files = []
unchanged_files = []

# Iterate through all review files in the input folder
for review_file in os.listdir(input_folder):
    # Construct the full path to the input and output files
    input_file_path = os.path.join(input_folder, review_file)
    output_file_path = os.path.join(output_folder, review_file)
    
    # Check if it's a valid text file
    if os.path.isfile(input_file_path) and review_file.endswith(".txt"):
        with open(input_file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
        
        # Filter out lines with ONLY coded symbols or ONLY a price
        cleaned_lines = [
            line for line in lines
            if line.strip() not in coded_symbols and not price_regex.match(line)
        ]

        # Check if any lines were removed
        if len(cleaned_lines) != len(lines):
            # Save the cleaned text to the output file
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(cleaned_lines)
            
            # Add to the list of updated files
            updated_files.append(review_file)
        else:
            # Save the unchanged file as is in the output folder
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)
            
            # Add to the list of unchanged files
            unchanged_files.append(review_file)

# Print final summary
print("\n--- Processing Summary ---")
print(f"Total files updated: {len(updated_files)}")
print(f"Total files unchanged: {len(unchanged_files)}")
if updated_files:
    print("\nFiles where deletions were made:")
    for file in updated_files:
        print(f" - {file}")
if unchanged_files:
    print("\nFiles saved without changes:")
    for file in unchanged_files:
        print(f" - {file}")


--- Processing Summary ---
Total files updated: 14995
Total files unchanged: 440

Files where deletions were made:
 - 19_(01)_1965_001.txt
 - 19_(01)_1965_002.txt
 - 19_(01)_1965_003.txt
 - 19_(01)_1965_004.txt
 - 19_(01)_1965_005.txt
 - 19_(01)_1965_006.txt
 - 19_(01)_1965_007.txt
 - 19_(01)_1965_008.txt
 - 19_(01)_1965_009.txt
 - 19_(01)_1965_010.txt
 - 19_(01)_1965_012.txt
 - 19_(01)_1965_013.txt
 - 19_(01)_1965_014.txt
 - 19_(01)_1965_015.txt
 - 19_(01)_1965_016.txt
 - 19_(01)_1965_017.txt
 - 19_(01)_1965_018.txt
 - 19_(01)_1965_019.txt
 - 19_(01)_1965_020.txt
 - 19_(01)_1965_021.txt
 - 19_(01)_1965_022.txt
 - 19_(01)_1965_023.txt
 - 19_(01)_1965_024.txt
 - 19_(01)_1965_025.txt
 - 19_(01)_1965_026.txt
 - 19_(01)_1965_027.txt
 - 19_(01)_1965_028.txt
 - 19_(01)_1965_029.txt
 - 19_(01)_1965_030.txt
 - 19_(01)_1965_031.txt
 - 19_(01)_1965_032.txt
 - 19_(01)_1965_033.txt
 - 19_(01)_1965_034.txt
 - 19_(01)_1965_035.txt
 - 19_(01)_1965_036.txt
 - 19_(01)_1965_037.txt
 - 19_(01)_1965_038.

### Remove reviewer abbreviations

In [30]:
# Define input and output folders
input_folder = "Reviews Processed4"  # Input folder containing review files
output_folder = "Reviews Processed5"  # Output folder for cleaned files
metadata_file = "Chunk 5 IDs Unique.xlsx"  # Excel file with metadata

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the metadata Excel file
df = pd.read_excel(metadata_file, dtype=str)

# Regex to match a 2-3 uppercase abbreviation at the end of the last line
abbreviation_regex = re.compile(r"[A-Z]{2,3}\.?$")
# Regex to match reviewer abbreviation anywhere in a line
reviewer_regex_template = r"\b{}\b"

# Track updated files
updated_files = []

# Function to clean reviewer abbreviation from lines
def remove_reviewer(lines, reviewer_abbreviation):
    updated_lines = []
    reviewer_removed = False
    reviewer_regex = re.compile(reviewer_regex_template.format(re.escape(reviewer_abbreviation)))

    for line in lines:
        if not reviewer_removed and reviewer_regex.search(line):
            line = reviewer_regex.sub("", line)  # Remove the abbreviation
            reviewer_removed = True  # Ensure only one match is removed
        updated_lines.append(line)
    return updated_lines, reviewer_removed

# Process each file in the folder
for review_file in os.listdir(input_folder):
    # Construct the full path to the input and output files
    input_file_path = os.path.join(input_folder, review_file)
    output_file_path = os.path.join(output_folder, review_file)
    
    # Check if it's a valid text file
    if os.path.isfile(input_file_path) and review_file.endswith(".txt"):
        with open(input_file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Ensure only non-empty lines are considered
        non_empty_lines = [line.strip() for line in lines if line.strip()]
        if not non_empty_lines:
            # If the file is empty or only whitespace, save it as-is
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)
            continue

        # Extract the last non-empty line
        last_line = non_empty_lines[-1]

        # Check if the last line contains a 2-3 uppercase abbreviation at the end
        if abbreviation_regex.search(last_line[-5:]):
            # Remove the abbreviation from the last line
            cleaned_last_line = abbreviation_regex.sub("", last_line).rstrip()

            # Replace the last non-empty line in the original lines
            for i in range(len(lines) - 1, -1, -1):
                if lines[i].strip():  # Find the last non-empty line in the original list
                    lines[i] = cleaned_last_line + "\n"
                    break

        # Extract the corresponding Reviewer value from the metadata file
        file_id = os.path.splitext(review_file)[0]  # Extract ID from filename
        matching_row = df[df['ID'] == file_id]

        reviewer_removed = False
        if not matching_row.empty and pd.notna(matching_row.iloc[0]['Reviewer']):
            reviewer_abbreviation = matching_row.iloc[0]['Reviewer'].strip()
            # Remove reviewer abbreviation from lines
            lines, reviewer_removed = remove_reviewer(lines, reviewer_abbreviation)

        # Save the updated file
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.writelines(lines)

        # Track the file if any modification was made
        if abbreviation_regex.search(last_line[-5:]) or reviewer_removed:
            updated_files.append(review_file)

# Print final summary
print("\n--- Processing Summary ---")
print(f"Total files updated (abbreviations or reviewers removed): {len(updated_files)}")
if updated_files:
    print("\nFiles with modifications:")
    for file in updated_files:
        print(f" - {file}")
else:
    print("No files had abbreviations or reviewers removed.")


--- Processing Summary ---
Total files updated (abbreviations or reviewers removed): 188

Files with modifications:
 - 27_(06)_1974_023.txt
 - 30_(07)_1977_044.txt
 - 35_(11)_1982_032.txt
 - 40_(01)_1986_003.txt
 - 40_(01)_1986_012.txt
 - 40_(01)_1986_032.txt
 - 40_(01)_1986_038.txt
 - 40_(01)_1986_041.txt
 - 40_(01)_1986_043.txt
 - 40_(01)_1986_046.txt
 - 40_(01)_1986_052.txt
 - 40_(01)_1986_065.txt
 - 40_(02)_1986_005.txt
 - 40_(02)_1986_008.txt
 - 40_(02)_1986_020.txt
 - 40_(02)_1986_025.txt
 - 40_(02)_1986_026.txt
 - 40_(02)_1986_029.txt
 - 40_(02)_1986_034.txt
 - 40_(02)_1986_043.txt
 - 40_(02)_1986_057.txt
 - 40_(02)_1986_060.txt
 - 40_(02)_1986_068.txt
 - 40_(03)_1986_001.txt
 - 40_(03)_1986_003.txt
 - 40_(03)_1986_004.txt
 - 40_(03)_1986_012.txt
 - 40_(03)_1986_018.txt
 - 40_(03)_1986_019.txt
 - 40_(03)_1986_022.txt
 - 40_(03)_1986_023.txt
 - 40_(03)_1986_034.txt
 - 40_(03)_1986_042.txt
 - 40_(03)_1986_043.txt
 - 40_(03)_1986_051.txt
 - 40_(03)_1986_053.txt
 - 40_(03)_1986_061

### Removing lines which ONLY say "Reviewed from galleys"

In [31]:
# Define the input and output folders
input_folder = "Reviews Processed5"  # Input folder
output_folder = "Reviews Processed6"  # Output folder for cleaned files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Regex to match a line that says "Reviewed from galleys" (case-insensitive, with optional punctuation)
reviewed_regex = re.compile(r"^\s*reviewed from galleys[\.\!\?]*\s*$", re.IGNORECASE)

# Track updated files
updated_files = []

# Process each file in the folder
for review_file in os.listdir(input_folder):
    # Construct the full path to the input and output files
    input_file_path = os.path.join(input_folder, review_file)
    output_file_path = os.path.join(output_folder, review_file)
    
    # Check if it's a valid text file
    if os.path.isfile(input_file_path) and review_file.endswith(".txt"):
        with open(input_file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Filter out lines that match the regex
        cleaned_lines = [line for line in lines if not reviewed_regex.match(line.strip())]

        # Check if any lines were removed
        if len(cleaned_lines) != len(lines):
            # Save the cleaned text to the output file
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(cleaned_lines)
            
            # Add to the list of updated files
            updated_files.append(review_file)
        else:
            # Save the unchanged file as is in the output folder
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)

# Print final summary
print("\n--- Processing Summary ---")
print(f"Total files updated (lines removed): {len(updated_files)}")
if updated_files:
    print("\nFiles with removed lines:")
    for file in updated_files:
        print(f" - {file}")
else:
    print("No files had matching lines removed.")


--- Processing Summary ---
Total files updated (lines removed): 61

Files with removed lines:
 - 39_(02)_1985_071.txt
 - 39_(05)_1986_015.txt
 - 39_(07)_1986_003.txt
 - 39_(07)_1986_022.txt
 - 39_(07)_1986_030.txt
 - 39_(07)_1986_050.txt
 - 39_(08)_1986_026.txt
 - 39_(08)_1986_040.txt
 - 39_(09)_1986_045.txt
 - 39_(09)_1986_047.txt
 - 39_(10)_1986_062.txt
 - 39_(10)_1986_068.txt
 - 39_(11)_1986_016.txt
 - 39_(11)_1986_033.txt
 - 40_(01)_1986_029.txt
 - 40_(01)_1986_063.txt
 - 40_(02)_1986_003.txt
 - 40_(02)_1986_004.txt
 - 40_(02)_1986_006.txt
 - 40_(02)_1986_041.txt
 - 40_(02)_1986_042.txt
 - 40_(02)_1986_051.txt
 - 40_(02)_1986_054.txt
 - 40_(03)_1986_001.txt
 - 40_(03)_1986_004.txt
 - 40_(03)_1986_042.txt
 - 40_(04)_1986_001.txt
 - 40_(04)_1986_002.txt
 - 40_(04)_1986_004.txt
 - 40_(04)_1986_027.txt
 - 40_(04)_1986_030.txt
 - 40_(04)_1986_060.txt
 - 40_(04)_1986_063.txt
 - 40_(05)_1987_004.txt
 - 40_(05)_1987_033.txt
 - 40_(05)_1987_064.txt
 - 40_(06)_1987_003.txt
 - 40_(06)_1987_0

### Removed lines with "reviewed from galleys" followed by a price

In [32]:
import os
import re

# Define input and output folders
input_folder = "Reviews Processed6"  # Input folder
output_folder = "Reviews Processed7"  # Output folder

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Partial match criteria (lowercase for case-insensitivity)
search_phrase = "reviewed from galleys"
price_symbol = "$"

# Function to process a single file
def process_file(input_path, output_path, filename):
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    cleaned_lines = []  # Store cleaned lines
    removed_lines = []  # Track removed lines
    
    # Process each line
    for line in lines:
        # Convert line to lowercase for case-insensitive comparison
        line_lower = line.lower()
        if search_phrase in line_lower and price_symbol in line_lower:
            removed_lines.append(line.strip())  # Log the removed line
            continue  # Skip this line
        cleaned_lines.append(line)
    
    # Write the cleaned lines to the output file
    with open(output_path, 'w', encoding='utf-8') as file:
        file.writelines(cleaned_lines)

    return removed_lines

# Main loop to process all files
files_with_removals = 0
all_removed_lines = {}

for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        input_file_path = os.path.join(input_folder, filename)
        output_file_path = os.path.join(output_folder, filename)
        
        removed_lines = process_file(input_file_path, output_file_path, filename)
        if removed_lines:
            files_with_removals += 1
            all_removed_lines[filename] = removed_lines
            print(f"Processed file: {filename} | Removed lines: {len(removed_lines)}")

# Final summary
print("\n--- Processing Summary ---")
print(f"Total files with removals: {files_with_removals}")
if files_with_removals:
    print("\nDetails of Removed Lines:")
    for file, lines in all_removed_lines.items():
        print(f"\nFile: {file}")
        for line in lines:
            print(f" - {line}")

Processed file: 39_(01)_1985_046.txt | Removed lines: 1
Processed file: 39_(08)_1986_010.txt | Removed lines: 1
Processed file: 39_(08)_1986_017.txt | Removed lines: 1
Processed file: 39_(08)_1986_050.txt | Removed lines: 1
Processed file: 40_(03)_1986_011.txt | Removed lines: 1
Processed file: 40_(06)_1987_001.txt | Removed lines: 1
Processed file: 40_(07)_1987_002.txt | Removed lines: 1
Processed file: 40_(07)_1987_021.txt | Removed lines: 1
Processed file: 40_(07)_1987_047.txt | Removed lines: 1
Processed file: 40_(07)_1987_055.txt | Removed lines: 1
Processed file: 40_(08)_1987_011.txt | Removed lines: 1
Processed file: 40_(09)_1987_009.txt | Removed lines: 1
Processed file: 40_(09)_1987_041.txt | Removed lines: 1
Processed file: 40_(10)_1987_066.txt | Removed lines: 1
Processed file: 40_(11)_1987_022.txt | Removed lines: 1
Processed file: 40_(11)_1987_056.txt | Removed lines: 1

--- Processing Summary ---
Total files with removals: 16

Details of Removed Lines:

File: 39_(01)_1985

### Remove lines which contain illustration metadata

In [33]:
# Define input and output folders
input_folder = "Reviews Processed7"  # Input folder containing review files
output_folder = "Reviews Processed8"  # Output folder for cleaned files

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Patterns to match
patterns = [
    r"illus\. by",         # Matches 'illus. by'
    r"trade ed\.",         # Matches 'Trade ed.'
    r"library ed\.",       # Matches 'Library ed.'
    r"paper ed\."          # Matches 'Paper ed.'
]

# Combine patterns into a single regex (case-insensitive)
combined_pattern = re.compile("|".join(patterns), re.IGNORECASE)

# Track updated files
updated_files = []

# Function to process files
def process_file(input_path, output_path):
    updated = False
    with open(input_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    cleaned_lines = []
    for line in lines:
        # If a line matches the pattern, skip it
        if combined_pattern.search(line.lower()):
            updated = True  # Mark as updated
            continue
        cleaned_lines.append(line)

    # Save the cleaned file
    with open(output_path, 'w', encoding='utf-8') as file:
        file.writelines(cleaned_lines)

    return updated

# Process each file
for review_file in os.listdir(input_folder):
    input_file_path = os.path.join(input_folder, review_file)
    output_file_path = os.path.join(output_folder, review_file)

    # Check if it's a valid text file
    if os.path.isfile(input_file_path) and review_file.endswith(".txt"):
        if process_file(input_file_path, output_file_path):
            updated_files.append(review_file)
        else:
            # Save original file if no updates were made
            with open(input_file_path, 'r', encoding='utf-8') as infile:
                with open(output_file_path, 'w', encoding='utf-8') as outfile:
                    outfile.writelines(infile.readlines())

# Print final summary
print("\n--- Processing Summary ---")
print(f"Total files updated (lines removed): {len(updated_files)}")
if updated_files:
    print("\nFiles with lines removed:")
    for file in updated_files:
        print(f" - {file}")
else:
    print("No files had lines removed.")


--- Processing Summary ---
Total files updated (lines removed): 2151

Files with lines removed:
 - 19_(01)_1965_003.txt
 - 19_(01)_1965_004.txt
 - 19_(01)_1965_013.txt
 - 19_(01)_1965_019.txt
 - 19_(01)_1965_022.txt
 - 19_(01)_1965_028.txt
 - 19_(01)_1965_036.txt
 - 19_(01)_1965_049.txt
 - 19_(01)_1965_054.txt
 - 19_(01)_1965_059.txt
 - 19_(01)_1965_062.txt
 - 19_(01)_1965_064.txt
 - 19_(01)_1965_071.txt
 - 19_(01)_1965_077.txt
 - 19_(01)_1965_085.txt
 - 19_(01)_1965_086.txt
 - 19_(01)_1965_089.txt
 - 19_(01)_1965_093.txt
 - 19_(02)_1965_015.txt
 - 19_(02)_1965_026.txt
 - 19_(02)_1965_045.txt
 - 19_(02)_1965_057.txt
 - 19_(03)_1965_008.txt
 - 19_(03)_1965_009.txt
 - 19_(03)_1965_011.txt
 - 19_(03)_1965_021.txt
 - 19_(03)_1965_026.txt
 - 19_(03)_1965_033.txt
 - 19_(03)_1965_035.txt
 - 19_(03)_1965_045.txt
 - 19_(04)_1965_007.txt
 - 19_(04)_1965_008.txt
 - 19_(04)_1965_021.txt
 - 19_(04)_1965_022.txt
 - 19_(04)_1965_024.txt
 - 19_(04)_1965_025.txt
 - 19_(04)_1965_026.txt
 - 19_(04)_1965

### Remove lines which contain values from both Author and Title column

In [34]:
# Define input and output folders
input_folder = "Reviews Processed8"  # Input folder containing review files
output_folder = "Reviews Processed9"  # Output folder for cleaned files
metadata_file = "Chunk 5 IDs Unique.xlsx"  # Excel file with metadata

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the metadata Excel file
df = pd.read_excel(metadata_file, dtype=str)

# Helper function to clean and normalize text
def clean_text(text):
    return re.sub(r'\s+', ' ', str(text).strip()).lower()

# Function to check if both Author and Title are in the line
def contains_author_and_title(line, author, title):
    return clean_text(author) in clean_text(line) and clean_text(title) in clean_text(line)

# Track files with lines removed
updated_files = []

# Process each file in the input folder
for review_file in os.listdir(input_folder):
    # Construct the full path to the input and output files
    input_file_path = os.path.join(input_folder, review_file)
    output_file_path = os.path.join(output_folder, review_file)

    # Check if it's a valid text file
    if os.path.isfile(input_file_path) and review_file.endswith(".txt"):
        with open(input_file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Extract the corresponding metadata for this file
        file_id = os.path.splitext(review_file)[0]  # Extract ID from filename
        matching_row = df[df['ID'] == file_id]

        if matching_row.empty:
            # If no metadata is found, save the file unchanged
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)
            continue

        # Get Author and Title values
        author = matching_row.iloc[0]['Author']
        title = matching_row.iloc[0]['Title']

        # Clean the file by removing lines with both Author and Title
        cleaned_lines = []
        line_removed = False

        for line in lines:
            if contains_author_and_title(line, author, title):
                line_removed = True  # Mark that a line was removed
                continue  # Skip this line
            cleaned_lines.append(line)

        # Save the cleaned file
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.writelines(cleaned_lines)

        # Track the file if lines were removed
        if line_removed:
            updated_files.append(review_file)

# Print final summary
print("\n--- Processing Summary ---")
print(f"Total files updated (lines with Author and Title removed): {len(updated_files)}")
if updated_files:
    print("\nFiles with modifications:")
    for file in updated_files:
        print(f" - {file}")
else:
    print("No files had lines removed.")


--- Processing Summary ---
Total files updated (lines with Author and Title removed): 2205

Files with modifications:
 - 19_(01)_1965_008.txt
 - 19_(01)_1965_014.txt
 - 19_(01)_1965_015.txt
 - 19_(01)_1965_017.txt
 - 19_(01)_1965_020.txt
 - 19_(01)_1965_021.txt
 - 19_(01)_1965_024.txt
 - 19_(01)_1965_029.txt
 - 19_(01)_1965_037.txt
 - 19_(01)_1965_038.txt
 - 19_(01)_1965_041.txt
 - 19_(01)_1965_043.txt
 - 19_(01)_1965_045.txt
 - 19_(01)_1965_046.txt
 - 19_(01)_1965_050.txt
 - 19_(01)_1965_053.txt
 - 19_(01)_1965_060.txt
 - 19_(01)_1965_061.txt
 - 19_(01)_1965_066.txt
 - 19_(01)_1965_069.txt
 - 19_(01)_1965_070.txt
 - 19_(01)_1965_073.txt
 - 19_(01)_1965_081.txt
 - 19_(01)_1965_082.txt
 - 19_(01)_1965_094.txt
 - 19_(02)_1965_001.txt
 - 19_(02)_1965_010.txt
 - 19_(02)_1965_016.txt
 - 19_(02)_1965_018.txt
 - 19_(02)_1965_022.txt
 - 19_(02)_1965_023.txt
 - 19_(02)_1965_033.txt
 - 19_(02)_1965_035.txt
 - 19_(02)_1965_040.txt
 - 19_(02)_1965_044.txt
 - 19_(02)_1965_047.txt
 - 19_(02)_1965_0

In [ ]:
import os
import pandas as pd
import re

# Define input and output folders
input_folder = "Reviews Processed9"  # Input folder containing review files
output_folder = "Reviews Processed10"  # Output folder for cleaned files
metadata_file = "Chunk 5 IDs Unique.xlsx"  # Excel file with metadata

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the metadata Excel file
df = pd.read_excel(metadata_file, dtype=str)

# Helper function to clean and normalize text
def clean_text(text):
    return re.sub(r'\s+', ' ', str(text).strip()).lower()

# Helper function to check for multiple metadata matches in a line
def contains_multiple_metadata(line, publisher, price_list, pages):
    match_count = 0
    line = clean_text(line)

    # Check Publisher
    if publisher and clean_text(publisher) in line:
        match_count += 1

    # Check Prices (split by commas)
    if price_list:
        for price in price_list:
            if clean_text(price) in line:
                match_count += 1
                break  # Only count once for any price match

    # Check Pages
    if pages and clean_text(pages) in line:
        match_count += 1

    # Return True if at least 2 metadata fields match
    return match_count >= 2

# Track files with lines removed
updated_files = []

# Process each file in the input folder
for review_file in os.listdir(input_folder):
    # Construct the full path to the input and output files
    input_file_path = os.path.join(input_folder, review_file)
    output_file_path = os.path.join(output_folder, review_file)

    # Check if it's a valid text file
    if os.path.isfile(input_file_path) and review_file.endswith(".txt"):
        with open(input_file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()

        # Extract the corresponding metadata for this file
        file_id = os.path.splitext(review_file)[0]  # Extract ID from filename
        matching_row = df[df['ID'] == file_id]

        if matching_row.empty:
            # If no metadata is found, save the file unchanged
            with open(output_file_path, 'w', encoding='utf-8') as file:
                file.writelines(lines)
            continue

        # Get Publisher, Price, and Pages values
        publisher = matching_row.iloc[0]['Publisher']
        price_list = clean_text(matching_row.iloc[0]['Price']).split(',') if pd.notna(matching_row.iloc[0]['Price']) else []
        pages = matching_row.iloc[0]['Pages'] if pd.notna(matching_row.iloc[0]['Pages']) else ""

        # Clean the file by removing lines with at least two metadata matches
        cleaned_lines = []
        line_removed = False

        for line in lines:
            if contains_multiple_metadata(line, publisher, price_list, pages):
                line_removed = True  # Mark that a line was removed
                continue  # Skip this line
            cleaned_lines.append(line)

        # Save the cleaned file
        with open(output_file_path, 'w', encoding='utf-8') as file:
            file.writelines(cleaned_lines)

        # Track the file if lines were removed
        if line_removed:
            updated_files.append(review_file)

# Print final summary
print("\n--- Processing Summary ---")
print(f"Total files updated (lines with multiple metadata removed): {len(updated_files)}")
if updated_files:
    print("\nFiles with modifications:")
    for file in updated_files:
        print(f" - {file}")
else:
    print("No files had lines removed.")

# Data Validation

## Count the number of "$" present in review texts

In [35]:
import os

# Define the folder containing the review files
reviews_folder = "Reviews Processed9"

# Initialize a list to store file names with the $ symbol
files_with_dollar = []

# Iterate through all review files in the folder
for review_file in os.listdir(reviews_folder):
    # Construct the full path to the review file
    file_path = os.path.join(reviews_folder, review_file)
    
    # Check if it's a valid text file
    if os.path.isfile(file_path) and review_file.endswith(".txt"):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        # Check if the $ symbol is present in the content
        if "$" in content:
            files_with_dollar.append(review_file)

# Print the results
print(f"Number of files containing the '$' symbol: {len(files_with_dollar)}")
print("\nFiles containing the '$' symbol:")
for file_name in files_with_dollar:
    print(file_name)

Number of files containing the '$' symbol: 122

Files containing the '$' symbol:
21_(06)_1968_011.txt
21_(06)_1968_050.txt
22_(04)_1968_010.txt
23_(10)_1970_049.txt
24_(09)_1971_034.txt
24_(11)_1971_029.txt
25_(02)_1971_051.txt
25_(05)_1972_034.txt
25_(10)_1972_007.txt
25_(10)_1972_042.txt
26_(02)_1972_036.txt
26_(07)_1973_008.txt
28_(01)_1974_048.txt
28_(04)_1974_012.txt
28_(07)_1975_020.txt
28_(10)_1975_019.txt
29_(06)_1976_047.txt
29_(09)_1976_014.txt
29_(09)_1976_015.txt
29_(09)_1976_038.txt
29_(11)_1976_040.txt
30_(04)_1976_029.txt
30_(05)_1977_036.txt
30_(10)_1977_025.txt
31_(02)_1977_054.txt
31_(04)_1977_018.txt
32_(08)_1979_035.txt
32_(09)_1979_024.txt
32_(10)_1979_011.txt
32_(11)_1979_064.txt
33_(04)_1979_025.txt
33_(06)_1980_051.txt
34_(09)_1981_043.txt
35_(06)_1982_011.txt
35_(06)_1982_040.txt
35_(07)_1982_049.txt
35_(08)_1982_016.txt
35_(11)_1982_042.txt
36_(01)_1982_019.txt
36_(02)_1982_036.txt
36_(04)_1982_037.txt
36_(04)_1982_039.txt
36_(05)_1983_057.txt
36_(07)_1983_014

## Count the number of empty review files

In [36]:
import os

# Define input folder containing review text files
input_folder = "Reviews Processed9"  # Replace with your folder name

# List to store names of empty files
empty_files = []

# Process each file in the folder
for filename in os.listdir(input_folder):
    file_path = os.path.join(input_folder, filename)
    
    # Check if it's a valid text file
    if os.path.isfile(file_path) and filename.endswith(".txt"):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read().strip()  # Read file and strip whitespace
        
        # Check if file is empty (no content or only whitespace)
        if not content:
            empty_files.append(filename)

# Print the results
print("\n--- Empty Files Summary ---")
print(f"Total empty files: {len(empty_files)}")
if empty_files:
    print("\nList of empty files:")
    for file in empty_files:
        print(f" - {file}")
else:
    print("No empty files found.")


--- Empty Files Summary ---
Total empty files: 12

List of empty files:
 - 22_(02)_1968_005.txt
 - 29_(06)_1976_026.txt
 - 29_(09)_1976_007.txt
 - 29_(10)_1976_005.txt
 - 32_(08)_1979_059.txt
 - 32_(11)_1979_012.txt
 - 33_(02)_1979_052.txt
 - 33_(03)_1979_017.txt
 - 33_(04)_1979_024.txt
 - 33_(05)_1980_028.txt
 - 37_(09)_1984_046.txt
 - 40_(05)_1987_022.txt
